In [64]:
import pandas as pd
import numpy as np

In [65]:
file = "data/exoplanet.eu_catalog.csv"

In [66]:
data = pd.read_csv(file, skiprows=0, usecols=['radius', 'orbital_period', 'ra', 'dec', 'mag_v', 
                                              'star_distance', 'star_radius', 'mag_i', 'mag_k',
                                             'semi_major_axis', 'eccentricity', 'tzero_tr'])

In [67]:
# We want to drop rows with null planet and stellar radii
# We also need the V-band magnitudes
data = data.dropna(subset={"radius", "star_radius", "mag_v"})
# We want only stars with mag_v ~< 12
data = data[data["mag_v"] < 12]

# Now compute transit depth for each
transit_depth = (0.10049*data["radius"] / data["star_radius"])**2
# Compute transit signature in magnitudes
transit_sig = -2.5 * np.log10(1 - transit_depth)
# Drop all rows where transit signature is < 0.008
data = data[transit_sig > 0.008]

In [68]:
len(data)

128

In [69]:
ra              = np.array(data['ra']/360*24 ) - 5
dec             = np.array(data['dec'])
radius          = np.array(data['radius'])
mag_v           = np.array(data['mag_v'])
star_distance   = np.array(data['star_distance'])
star_radius     = np.array(data['star_radius'])
mag_i           = np.array(data['mag_i'])
mag_k           = np.array(data['mag_k'])
semi_major_axis = np.array(data['semi_major_axis'])*149597870691
eccentricity    = np.array(data['eccentricity'])
period          = np.array(data['orbital_period'])
primary_transit = np.array(data['tzero_tr'])
# saving all data types individually

In [70]:
semi_minor_axis = []
velocity        = []
for i in range(len(ra)):
    semi_minor_axis.append(semi_major_axis[i]*np.sqrt(1-eccentricity[i]**2))
    velocity.append(2*np.pi*((semi_major_axis[i] + semi_minor_axis[i])/2)  /   (period[i]*3600*24))
#calculates semi minor axis and average orbital velocity in m/s
transit_time    = (star_radius*6.9571*10**8)/3600/velocity

In [71]:
dec1 = []
for i in range(len(ra)):
    if ra[i] < 0:
        ra[i] = ra[i] + 24
    if ra[i] > 6:
        dec1.append(abs(18-ra[i])*82/12-10)
    else:
        dec1.append((6+ra[i])*82/12-10)
#lower bound for visible stars

In [84]:
ObsDay = primary_transit
j = 0
for i in range(len(ra)):
    while ObsDay[i] < 2458732.5:
        ObsDay = ObsDay + period[i]*j 
        j      = j + 1
np.nanmax(ObsDay)

2463994.1981027597

In [86]:
data1= []
for i in range(len(data)):
    if  dec[i] > dec1[i] and transit_time[i] < 3 and ra[i] > 19 and ObsDay[i] < 2458761.5:
        data1.append([radius[i],ra[i]*360/24 ,dec[i],mag_v[i],mag_i[i],mag_k[i],star_distance[i],star_radius[i],
                     velocity[i]])
#writes all data we may need for stars that follow what we need

In [87]:
data1

[]

In [63]:
np.savetxt("kiran.csv", data1, delimiter=',')

In [38]:
data

,radius,orbital_period,semi_major_axis,eccentricity,ra,dec,mag_v,mag_i,mag_k,star_distance,star_radius
48,1.90000,4.230800,0.05200,0.0069,344.362500,20.768611,5.490,NaN,NaN,14.70,1.266
311,1.31900,4.465300,0.05561,0.0670,344.445833,38.675000,10.400,NaN,NaN,139.00,1.174
312,1.04500,3.722469,0.04390,0.0000,47.370833,30.673611,11.890,NaN,NaN,125.00,0.810
320,1.28900,2.775960,0.04130,0.0360,9.575000,42.463056,10.800,NaN,NaN,235.00,1.237
321,1.01000,10.338523,0.08820,0.3460,324.537500,30.488611,10.540,NaN,NaN,90.00,0.837
326,0.86700,2.875317,0.03610,0.0150,111.916667,24.336389,11.340,NaN,NaN,70.00,0.694
327,1.02400,4.124461,0.04940,0.2280,171.275000,41.028056,11.460,NaN,NaN,254.00,1.105
328,1.08000,3.212220,0.04140,0.0160,155.683333,50.128333,9.730,NaN,NaN,82.00,1.040
329,1.36800,1.212884,0.02320,0.1060,306.125000,16.762222,11.940,NaN,NaN,393.00,1.290
330,1.24200,3.355240,0.04650,0.0670,108.825000,14.262222,11.820,NaN,NaN,306.00,1.317


**Need to filter out planets that are invisible from Stony Brook this time of year.**